In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly" #interactive plots will be useful in this context
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import gensim
from sentence_transformers import SentenceTransformer, util
import pickle
import torch
from sklearn.cluster import KMeans

C:\Users\BothmK01\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
df = pd.read_csv('../datasets/df_cv.csv')
k31_full = pickle.load(open('../Model_Selection/k_31_full', 'rb'))

In [20]:
df_sample = df.sample(n=50, random_state=1)

In [22]:
df_sample.to_csv('sample.csv')

## Labeling
After acquiring a random sample of 50 skills from the CV data each labels cluster was labeled 0 for false and 1 for true. This was done outside of the coding environment and stored in csv 'sample_labeled.csv'. After labeling it is read into memory to compute basic statistics.

In [31]:
df_labeled = pd.read_csv('sample_labeled.csv', sep=';')

In [32]:
df_labeled

,Unnamed: 0,skill,Label,cluster
0,3969,strong skills in statistical methodologies su...,0,12
1,3300,"extensive experience in development of t-sql,...",1,1
2,4384,worked on resulting reports of the application.,1,23
3,3308,"proficient in system analysis, er/dimensional...",0,30
4,2890,created a wholesale portal with easy checkout...,1,25
5,3830,"programming languages: python, sql, r, matlab...",1,18
6,545,responsible for debugging and troubleshooting...,0,22
7,1812,experience in big data technologies like spar...,1,8
8,2596,- built an nlp system to automatically assess...,1,9
9,959,experienced in ensemble learning usingbagging...,1,6


In [33]:
df_labeled

,Unnamed: 0,skill,Label,cluster
0,3969,strong skills in statistical methodologies su...,0,12
1,3300,"extensive experience in development of t-sql,...",1,1
2,4384,worked on resulting reports of the application.,1,23
3,3308,"proficient in system analysis, er/dimensional...",0,30
4,2890,created a wholesale portal with easy checkout...,1,25
5,3830,"programming languages: python, sql, r, matlab...",1,18
6,545,responsible for debugging and troubleshooting...,0,22
7,1812,experience in big data technologies like spar...,1,8
8,2596,- built an nlp system to automatically assess...,1,9
9,959,experienced in ensemble learning usingbagging...,1,6


In [30]:
df_labeled['Label'].mean()

0.8163265306122449

In [35]:
from platform import python_version
print(python_version())


3.8.3
